In [ ]:
import os
import sys
import logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL
logging.getLogger('tensorflow').setLevel(logging.DEBUG)

try:
    from google.colab import drive
    drive.mount('/content/drive')
    !pip install -q ruamel.yaml
    !pip install -q tensorboard-plugin-profile
    project_path = '/content/drive/MyDrive/Colab Projects/QuantumFlow'
except:
    project_path = os.path.expanduser('~/QuantumFlow')

In [ ]:
os.chdir(project_path)
sys.path.append(project_path)

import numpy as np
import tensorflow as tf

import time
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import quantumflow

experiment = 'xdiff'
run_name = 'derivative'
epoch = None

base_dir = os.path.join(project_path, "experiments", experiment)
params = quantumflow.utils.load_yaml(os.path.join(base_dir, 'hyperparams.yaml'))[run_name]
run_dir = os.path.join(base_dir, run_name)

In [ ]:
dataset_train = quantumflow.instantiate(params['dataset_train'], run_dir=run_dir)
dataset_train.build()
h =  dataset_train.h
x =  dataset_train.x

dataset_validate = quantumflow.instantiate(params['dataset_validate'], run_dir=run_dir)
dataset_validate.build()

In [ ]:
_ = quantumflow.get_class(params['model']['class'])
model = tf.keras.models.load_model(os.path.join(run_dir, 'saved_model'))
if epoch is not None: _ = model.load_weights(os.path.join(run_dir, params['checkpoint']['filename'].format(epoch=epoch)))

In [ ]:
preview = 5
figsize = (20, 3)
dpi = None
kcalmol_per_hartree = 627.5094738898777

M = 1
N = 2
n_basis = 20
dtype = np.float32
step_size = 0.00001

steps = 10000
record_steps = max(100, steps//1000)
record_preview = preview

initial_density = np.repeat(np.expand_dims(np.mean(dataset_validate.density, axis=0), axis=0), dataset_validate.dataset_size, axis=0)
# initial_density = normalize_density(np.repeat(np.sin(x[np.newaxis, :]*np.pi)**2, M, axis=0))*params['N']
# initial_density = dataset.density

potential = dataset_validate.potential[:M]
initial_pseudo = np.sqrt(initial_density[:M])

class vWModel():
    @tf.custom_gradient
    def __call__(self, pseudo):
        def grad(kinetic_energy, kinetic_energy_density):
            return -h*quantumflow.utils.laplace_five_point(pseudo, h)
        kinetic_energy_density = 1/2*tf.square(quantumflow.utils.derivative_five_point(pseudo, h))
        return {'kinetic_energy': quantumflow.utils.integrate(kinetic_energy_density, h), 'kinetic_energy_density': kinetic_energy_density}, grad

vW_model = vWModel()

zero_model = lambda features: {'kinetic_energy': 0.0*quantumflow.utils.integrate(features['density'], h)}


project_basis = np.sqrt(2)*np.sin(x[np.newaxis, :]*np.pi*np.arange(1, n_basis+1, dtype=dtype)[:, np.newaxis])

def project(value): 
    return quantumflow.utils.normalize(h*tf.matmul(tf.matmul(project_basis, value, transpose_b=True), project_basis, transpose_a=True), h)*np.sqrt(N)


In [ ]:
@tf.function
def step_fn(pseudo, potential):
    density = tf.math.square(pseudo)
    potential_energy = quantumflow.utils.integrate(potential*density, h)

    with tf.GradientTape() as tape:
        tape.watch(density)
        model_kinetic_energy = model({'density': tf.cast(density, dtype=tf.float32)})['kinetic_energy']
        
    original_model_derivative = 1/h*tape.gradient(model_kinetic_energy, density)
    model_kinetic_energy = tf.cast(model_kinetic_energy, dtype=dtype)
    original_model_derivative = tf.cast(original_model_derivative, dtype=dtype)

    model_derivative = original_model_derivative*2*pseudo

    with tf.GradientTape() as tape:
        tape.watch(pseudo)
        vW_kinetic_energy = vW_model(pseudo)['kinetic_energy']

    vW_derivative = 1/h*tape.gradient(vW_kinetic_energy, pseudo)

    derivative = model_derivative + vW_derivative
    kinetic_energy = model_kinetic_energy + vW_kinetic_energy

    vW_mu = 1/float(N)*(kinetic_energy + potential_energy)
    potential_derivative = 2*pseudo*(potential - tf.expand_dims(vW_mu, axis=-1))
    functional_derivative = derivative + potential_derivative
    fd = functional_derivative[:, 2:-2]
    functional_derivative = tf.concat([3*fd[:, 0:1] - 2*fd[:, 1:2], 2*fd[:, 0:1] - fd[:, 1:2], fd, 2*fd[:, -1:] - fd[:, -2:-1], 3*fd[:, -1:] - 2*fd[:, -2:-1]], axis=1)

    new_pseudo = pseudo - step_size*functional_derivative

    new_pseudo = project(new_pseudo)

    return new_pseudo, functional_derivative, original_model_derivative, vW_derivative, potential_derivative, kinetic_energy, potential_energy, vW_mu

recordings = {}
for key in ['pseudo', 'model_derivative', 'vW_derivative', 'functional_derivative']:
    recordings[key] = []

pseudo = initial_pseudo
print('/', steps)
for step in range(steps+1):
    pseudo, functional_derivative, model_derivative, vW_derivative, potential_derivative, kinetic_energy, potential_energy, vW_mu = step_fn(pseudo, potential)
    pseudo = tf.debugging.check_numerics(pseudo, message='Invalid value encountered, step {}.'.format(step+1))

    if step % record_steps == 0: 
        print(step, end=' ')
        recordings['pseudo'].append(pseudo.numpy())
        recordings['model_derivative'].append(model_derivative[:record_preview].numpy())
        recordings['vW_derivative'].append(vW_derivative[:record_preview].numpy())
        recordings['functional_derivative'].append(functional_derivative[:record_preview].numpy())
print()

In [ ]:
plt.figure(figsize=(20, 3), dpi=dpi)
plt.plot(x, np.transpose(dataset_validate.density[:1]), 'k:')
plt.plot(x, np.transpose(pseudo[:preview]**2))
plt.legend(['Target', 'Iterative'])
plt.show()

In [ ]:
plt.figure(figsize=(20, 3), dpi=dpi)
plt.plot(x, tf.transpose(pseudo[:preview]))
plt.legend(['initial value'])
plt.show()

pseudo, functional_derivative, model_derivative, vW_derivative, potential_derivative, kinetic_energy, potential_energy, vW_mu = step_fn(pseudo, potential)

plt.figure(figsize=(20, 3), dpi=dpi)
plt.plot(x, tf.transpose(model_derivative[:preview]))
plt.legend(['model_derivative'])
plt.show()

plt.figure(figsize=(20, 3), dpi=dpi)
plt.plot(x, tf.transpose(vW_derivative[:preview]))
plt.legend(['vW_derivative'])
plt.show()

plt.figure(figsize=(20, 3), dpi=dpi)
plt.plot(x, tf.transpose(potential_derivative[:preview]))
plt.legend(['potential_derivative'])
plt.show()

plt.figure(figsize=(20, 3), dpi=dpi)
plt.plot(x, tf.transpose(functional_derivative[:preview]))
plt.legend(['functional_derivative'])
plt.show()

In [ ]:
if value is None:
    value = dataset.pseudo if self.pseudo else dataset.density

assert dataset.dataset_size % batch_size == 0, "can't divide dataset into equal parts: {} % {} != 0".format(dataset.dataset_size, batch_size)
kinetic_energy_list = []
model_derivative_list = []
functional_derivative_list = []
for batch in range(dataset.dataset_size // batch_size):
    _, functional_derivative, model_derivative, _, _, kinetic_energy, _, _ = self.step(value[batch*batch_size:(batch+1)*batch_size], dataset.potential[batch*batch_size:(batch+1)*batch_size])
    kinetic_energy_list.append(kinetic_energy)
    model_derivative_list.append(model_derivative)
    functional_derivative_list.append(functional_derivative)

kinetic_energy = np.concatenate(kinetic_energy_list, axis=0)
model_derivative = np.concatenate(model_derivative_list, axis=0)
functional_derivative = np.concatenate(functional_derivative_list, axis=0)

kenergies_err = (kinetic_energy - dataset.kinetic_energy)*kcalmol_per_hartree
kenergies_abs_err = np.abs(kenergies_err)
print('Kinetic Energy:')
print('MAE:', np.nanmean(kenergies_abs_err))
print('AE_std:', np.std(kenergies_abs_err))
print('AE_max:', np.max(kenergies_abs_err))

plt.figure(figsize=figsize, dpi=dpi)
plt.hist(kenergies_err, bins=100)
plt.yscale('log', nonposy='clip')
plt.title('Kinetic Energy Error')
plt.xlabel('kcal/mol')
plt.show()

if self.pseudo:
    model_derivative += dataset.vW_derivative
functional_err = (model_derivative - dataset.derivative)
functional_norm_err = integrate(np.abs(functional_err), dataset.h)*kcalmol_per_hartree

print('Functional Derivative:')
print('MAE:', np.nanmean(functional_norm_err))
print('AE_std:', np.std(functional_norm_err))
print('AE_max:', np.max(functional_norm_err))

plt.figure(figsize=figsize, dpi=dpi)
plt.hist(functional_norm_err, bins=100)
plt.yscale('log', nonposy='clip')
plt.title('Functional Derivative Error')
plt.xlabel('hartree')
plt.show()

# Train

In [ ]:
start = time.time()
for batch in range(dataset.dataset_size // batch_size):
    progress.description = str(batch+1) + '/' + str(dataset.dataset_size // batch_size)
    self_consistency_method.train(dataset.potential[batch*batch_size:(batch+1)*batch_size], value[batch*batch_size:(batch+1)*batch_size], progress=progress)
    recordings = self_consistency_method.get_recordings()
    
    save_hdf5(os.path.join(base_dir, run_name, 'recordings_{:05d}.hdf5'.format(batch)), datasets=recordings)

duration = time.time() - start
print('duration:', duration, 'seconds')

In [ ]:
recordings = {key:[] for key in params['record'].keys()}

for batch in range(dataset.dataset_size // batch_size):
    for key, value in load_hdf5(os.path.join(base_dir, run_name, 'recordings_{:05d}.hdf5'.format(batch))).items():
        recordings[key].append(value)

for key in recordings.keys():
    recordings[key] = np.concatenate(recordings[key], axis=1)

self_consistent_value = recordings['value'][-1]

self_consistency_method.plot_performance(dataset, value=self_consistent_value, batch_size=params['batch_size'])

if params['pseudo_wavefunction']:
    self_consistent_density = tf.square(self_consistent_value)
else:
    self_consistent_density = self_consistent_value

density_err = (self_consistent_density - dataset.density)
density_norm_err = integrate(np.abs(density_err), dataset.h)
print('Density:')
print('MAE:', np.nanmean(density_norm_err))
print('AE_std:', np.std(density_norm_err))
print('AE_max:', np.max(density_norm_err))

In [ ]:
preview = 5
plt.figure(figsize=figsize, dpi=dpi)
plt.plot(dataset.x, np.transpose(dataset.density[:preview]), label='exact density')
plt.plot(dataset.x, np.transpose(self_consistent_density[:preview]), 'k--', label='self-consistent density')
plt.plot(dataset.x, np.transpose(initial_density[0:1]), 'grey', label='inital density')
plt.legend()
plt.title('density convergence')
plt.show()

model_kinetic_energy = []
kinetic_energy = []
potential_energy = []
vW_kinetic_energy = []

model_kinetic_energy_density = []
vW_kinetic_energy_density = []

for value_step in recordings['value']:
    value_step = tf.constant(value_step)

    if params['model_pseudo_wavefunction']:
        model_input = {'pseudo': tf.cast(value_step, dtype=params['model_dtype'])} if params['pseudo_wavefunction'] else {'pseudo': tf.cast(tf.math.sqrt(value_step), dtype=params['model_dtype'])}
    else:
        model_input = {'density': tf.cast(tf.square(value_step), dtype=params['model_dtype'])} if params['pseudo_wavefunction'] else {'density': tf.cast(value_step, dtype=params['model_dtype'])}
    
    model_predictions = model(model_input)

    model_kinetic_energy_step = tf.cast(model_predictions['kinetic_energy'], dtype=params['dtype'])
    vW_predictions = vW_model(value_step)
    vW_kinetic_energy_step = vW_predictions['kinetic_energy']

    if 'kinetic_energy_density' in model_predictions: 
        model_kinetic_energy_density_step = tf.cast(model_predictions['kinetic_energy_density'], dtype=params['dtype'])
        vW_kinetic_energy_density_step = vW_predictions['kinetic_energy_density']
        plot_kinetic_energy_density = True
    else:
        plot_kinetic_energy_density = False

    if params['pseudo_wavefunction']:
        potential_energy_step = integrate(tf.square(value_step)*dataset.potential, params['dataset_info']['h'])
    else:
        potential_energy_step = integrate(value_step*dataset.potential, params['dataset_info']['h'])

    model_kinetic_energy.append(model_kinetic_energy_step.numpy())
    vW_kinetic_energy.append(vW_kinetic_energy_step.numpy())
    potential_energy.append(potential_energy_step.numpy())

    if plot_kinetic_energy_density: 
        model_kinetic_energy_density.append(model_kinetic_energy_density_step.numpy())
        vW_kinetic_energy_density.append(vW_kinetic_energy_density_step.numpy())

model_kinetic_energy = np.stack(model_kinetic_energy, axis=0)
vW_kinetic_energy = np.stack(vW_kinetic_energy, axis=0)
potential_energy = np.stack(potential_energy, axis=0)

if plot_kinetic_energy_density:
    model_kinetic_energy_density = np.stack(model_kinetic_energy_density, axis=0)
    vW_kinetic_energy_density = np.stack(vW_kinetic_energy_density, axis=0)

kinetic_energy = model_kinetic_energy + params['model_subtract_von_weizsaecker']*vW_kinetic_energy

kenergies_err = np.abs(tf.expand_dims(dataset.kinetic_energy, axis=0) - kinetic_energy)*kcalmol_per_hartree
kenergies_err_mean = np.mean(kenergies_err, axis=1)
kenergies_err_std = np.std(kenergies_err, axis=1)

plt.figure(figsize=figsize, dpi=dpi)
plt.fill_between(np.arange(len(kenergies_err_mean)), kenergies_err_mean - kenergies_err_std, kenergies_err_mean + kenergies_err_std, alpha=0.2)
plt.plot(kenergies_err_mean)
#plt.ylim([0, 10])
plt.title('kinetic energy error')
plt.show()

plt.figure(figsize=figsize, dpi=dpi)
plt.plot(model_kinetic_energy)

plt.title('model kinetic energy')
plt.show()

plt.figure(figsize=figsize, dpi=dpi)
plt.plot(vW_kinetic_energy)
plt.title('von Weizsäcker kinetic energy')
plt.show()

plt.figure(figsize=figsize, dpi=dpi)
plt.plot(potential_energy)
plt.title('potential energy')
plt.show()

plt.figure(figsize=figsize, dpi=dpi)
plt.plot(potential_energy + kinetic_energy)
plt.title('total energy')
plt.show()


In [ ]:
worst_index = np.argmax(kenergies_err[-1])
print(kenergies_err[-1][worst_index])

plt.figure(figsize=figsize, dpi=dpi)
plt.plot(dataset.x, np.transpose(dataset.density[worst_index:worst_index+1]), label='exact density')
plt.plot(dataset.x, np.transpose(self_consistent_density[worst_index:worst_index+1]), 'k--', label='self-consistent density')
plt.plot(dataset.x, np.transpose(initial_density[0:1]), 'grey', label='inital density')
plt.legend()
plt.title('density convergence')
plt.show()

if plot_kinetic_energy_density:
    plt.figure(figsize=figsize, dpi=dpi)
    plt.plot(dataset.x, np.transpose(model_kinetic_energy_density[-1, worst_index:worst_index+1]))
    plt.show()

    plt.figure(figsize=figsize, dpi=dpi)
    plt.plot(dataset.x, np.transpose(vW_kinetic_energy_density[-1, worst_index:worst_index+1]))
    plt.show()

In [ ]:
if plot_kinetic_energy_density:
    print('model_kinetic_energy_density')
    anim_plot(np.moveaxis(model_kinetic_energy_density, 2, 1), bar='Rendering')
    print('vW_kinetic_energy_density')
    anim_plot(np.moveaxis(vW_kinetic_energy_density, 2, 1), bar='Rendering')

In [ ]:
print('densities/pseudo-wavefunctions')
anim_plot(np.moveaxis(recordings['value'], 2, 1), bar='Rendering')
print('functional derivatives')
anim_plot(np.moveaxis(recordings['functional_derivative'], 2, 1), bar='Rendering')
print('model derivative')
anim_plot(np.moveaxis(recordings['model_derivative'], 2, 1), bar='Rendering')
print('von Weizsäcker derivative')
anim_plot(np.moveaxis(recordings['vW_derivative'], 2, 1), bar='Rendering')